## Rag with multi Data Source 

In [23]:

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [24]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=250)
wiki= WikipediaQueryRun(api_wrapper=api_wrapper)
print(wiki.name)

wikipedia


In [25]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

from langchain_openai import OpenAIEmbeddings # you can use OllamaEmbeddings or HuggingFaceEmbeddings as well
from langchain_text_splitters import RecursiveCharacterTextSplitter
import bs4  

loader = WebBaseLoader("https://ekaivakriti.com/")
docs = loader.load()
documents= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorDb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectorDb.as_retriever
retriever

<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x10e3a7650>>

In [26]:
from langchain.tools.retriever import create_retriever_tool
RetrieverTool = create_retriever_tool(
    retriever=retriever,
    name="WebRetriever",
    description="Useful for retrieving information from the web."
)
RetrieverTool

Tool(name='WebRetriever', description='Useful for retrieving information from the web.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x12888d580>, retriever=<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x10e3a7650>>, document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x12888d8a0>, retriever=<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x10e3a7650>>, document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'))

## Arxiv Tool(Just like Wiki Tool)

In [27]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))

In [28]:
tools = [wiki, arxiv, RetrieverTool]

In [29]:
from dotenv import load_dotenv
load_dotenv()
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

## Prompt Template

In [30]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Agents

In [31]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt) #chain
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [32]:
## To run Agent We need Agent Executor
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executer
# Run the agent with a query
# agent_executer.invoke({"input": "Tell me about Ekaivakriti?"})

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [33]:
agent_executer.invoke({"input": "What is the latest research on AI in healthcare?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'AI in healthcare'}`


Published: 2025-04-30
Title: SoK: Security and Privacy Risks of Healthcare AI
Authors: Yuanhaur Chang, Han Liu, Chenyang Lu, Ning Zhang
Summary: The integration of artificial intelligence (AI) and machine learning (ML)
into healthcare systems holds gThe latest research on AI in healthcare is a paper titled "SoK: Security and Privacy Risks of Healthcare AI" by Yuanhaur Chang, Han Liu, Chenyang Lu, and Ning Zhang. The paper discusses the integration of artificial intelligence (AI) and machine learning (ML) into healthcare systems and the associated security and privacy risks. It was published on April 30, 2025.

> Finished chain.


{'input': 'What is the latest research on AI in healthcare?',
 'output': 'The latest research on AI in healthcare is a paper titled "SoK: Security and Privacy Risks of Healthcare AI" by Yuanhaur Chang, Han Liu, Chenyang Lu, and Ning Zhang. The paper discusses the integration of artificial intelligence (AI) and machine learning (ML) into healthcare systems and the associated security and privacy risks. It was published on April 30, 2025.'}

In [34]:
agent_executer.invoke({"input": "Tell me about Ekaivakriti?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Ekaivakriti'}`


No good Wikipedia Search Result was foundI couldn't find any information about Ekaivakriti on Wikipedia. Would you like me to search for information about Ekaivakriti using a different source?

> Finished chain.


{'input': 'Tell me about Ekaivakriti?',
 'output': "I couldn't find any information about Ekaivakriti on Wikipedia. Would you like me to search for information about Ekaivakriti using a different source?"}